# Formatos comunes en las respuestas HTTP

Las bases datos biológicas, y otros servicios online, usan tipicamente poco formatos para devolver los datos:

- XML
- JSON
- Texto tabular


## XML - Extensible Markup Language

Es un formato de texto que permite representar datos de formar jerarquica y arbitrario.
No está pensado para ser leido directamente por un usuario.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<ResultadoDeBusqueda>
    <Proteinas cantidad="2">
        <Protein>
            <Nombre>
                FUS
            </Nombre>
            <ProteinID>
                Q6IBQ5
            </ProteinID>
            <Length>
                526
            </Length>
        </Protein>
        <Protein>
            <Nombre>
                FIBP
            </Nombre>
            <ProteinID>
                Q6IBQ3
            </ProteinID>
            <Length>
                364
            </Length>
        </Protein>
    </Proteinas>
    <NumeroDeResultados>
        2
    </NumeroDeResultados>
</ResultadoDeBusqueda>
```

En R hay librarias "standard" para procesar estos archivos.

- XML
- xml2

Con ambas se pueden hacer las mismas cosas.

Algunos paquetes que nos permiten acceder a las bases de datos pueden devolver
resultados generados con una de ellas, miestras que otras pueden usar otro.

### Libraria XML

In [2]:
library(XML)

In [3]:

xml_content <- '<?xml version="1.0" encoding="UTF-8"?>
<ResultadoDeBusqueda>
<Proteinas cantidad="2">
<Protein>
<Nombre>FUS</Nombre>
<ProteinID>Q6IBQ5</ProteinID>
<Longitud>526</Longitud>
</Protein>
<Protein>
<Nombre>FIBP</Nombre>
<ProteinID>Q6IBQ3</ProteinID>
<Longitud>364</Longitud>
</Protein>
</Proteinas>
<NumeroDeResultados>2</NumeroDeResultados>
</ResultadoDeBusqueda>'

# Lee el Texto en formato XML
xml_document <- xmlParse(xml_content)
print(xml_document)
print(class(xml_document))

<?xml version="1.0" encoding="UTF-8"?>
<ResultadoDeBusqueda>
  <Proteinas cantidad="2">
    <Protein>
      <Nombre>FUS</Nombre>
      <ProteinID>Q6IBQ5</ProteinID>
      <Longitud>526</Longitud>
    </Protein>
    <Protein>
      <Nombre>FIBP</Nombre>
      <ProteinID>Q6IBQ3</ProteinID>
      <Longitud>364</Longitud>
    </Protein>
  </Proteinas>
  <NumeroDeResultados>2</NumeroDeResultados>
</ResultadoDeBusqueda>
 
[1] "XMLInternalDocument" "XMLAbstractDocument"


In [6]:
# Si quiero acceder a un nodo particular puedo hacer esto:
my_node <- getNodeSet(xml_document, "//Protein")
print(my_node)
print(my_node[[1]])

[[1]]
<Protein>
  <Nombre>FUS</Nombre>
  <ProteinID>Q6IBQ5</ProteinID>
  <Longitud>526</Longitud>
</Protein> 

[[2]]
<Protein>
  <Nombre>FIBP</Nombre>
  <ProteinID>Q6IBQ3</ProteinID>
  <Longitud>364</Longitud>
</Protein> 

attr(,"class")
[1] "XMLNodeSet"
<Protein>
  <Nombre>FUS</Nombre>
  <ProteinID>Q6IBQ5</ProteinID>
  <Longitud>526</Longitud>
</Protein> 


In [12]:
# Si quiero recuperar el nombre de la proteina de uno de los nodos puedo hacer:

secondProteinName <- xmlValue(getNodeSet(xml_document, "//Nombre")[2])
secondProteinName

[1] "FIBP"

In [14]:
# Si quiero extraer todos los nombres de la proteinas puedo hacer esto:
proteinNames <- xpathApply(
  xml_document,
  "//Nombre",
  xmlValue
)
proteinNames

[[1]]
[1] "FUS"

[[2]]
[1] "FIBP"

In [26]:
# Si quiero extrart varios valores de un nodo puedo hacer esto:
proteinNames <- xpathApply(
  xml_document,
  "//Protein",
  function(x) {
    return(
      c(
        xpathSApply(x, "./Nombre", xmlValue),
        xpathSApply(x, "./ProteinID", xmlValue)
      )
    )
  }
)
print(proteinNames)

[[1]]
[1] "FUS"    "Q6IBQ5"

[[2]]
[1] "FIBP"   "Q6IBQ3"



In [4]:
# Otra opcion para hacer lo mismo, pero mas complicado....
extract_protein_info <- function(node) {
  c(
    Nombre = xmlValue(getNodeSet(node, "./Nombre")[[1]]),
    ProteinID = xmlValue(getNodeSet(node, "./ProteinID")[[1]])
  )
}

# Apply the custom function to each "Protein" node
protein_list <- xpathApply(
  xml_document,
  "//Protein",
  extract_protein_info
)

# Check the result
print(protein_list)


[[1]]
   Nombre ProteinID 
    "FUS"  "Q6IBQ5" 

[[2]]
   Nombre ProteinID 
   "FIBP"  "Q6IBQ3" 



In [35]:
# Tambien es posible convertir el contenido XML a una lista de R

xml_list <- xmlToList(xml_document)
xml_list

print(xml_list$Proteinas[2]$Protein)

$Proteinas
$Proteinas$Protein
$Proteinas$Protein$Nombre
[1] "FUS"

$Proteinas$Protein$ProteinID
[1] "Q6IBQ5"

$Proteinas$Protein$Longitud
[1] "526"


$Proteinas$Protein
$Proteinas$Protein$Nombre
[1] "FIBP"

$Proteinas$Protein$ProteinID
[1] "Q6IBQ3"

$Proteinas$Protein$Longitud
[1] "364"


$Proteinas$.attrs
cantidad 
     "2" 


$NumeroDeResultados
[1] "2"

$Nombre
[1] "FIBP"

$ProteinID
[1] "Q6IBQ3"

$Longitud
[1] "364"



In [38]:
# Tambien es posible convertir el contenido XML o parte del contenido en un
# dataframe.
xml_dataframe <- xmlToDataFrame(getNodeSet(xml_document, "//Protein"))
xml_dataframe

Nombre,ProteinID,Longitud
<chr>,<chr>,<chr>
FUS,Q6IBQ5,526
FIBP,Q6IBQ3,364


In [41]:
# Puedo extraer atributos de un nodo:

xmlAttrs(
  getNodeSet(xml_document, "//Proteinas")[[1]]
)

cantidad 
     "2"

### Libraria xml2

In [ ]:

library(xml2)

In [ ]:
xml_content <- '<?xml version="1.0" encoding="UTF-8"?>
<ResultadoDeBusqueda>
<Proteinas>
<Protein>
<Nombre>FUS</Nombre>
<ProteinID>Q6IBQ5</ProteinID>
<Longitud>526</Longitud>
</Protein>
<Protein>
<Nombre>FIBP</Nombre>
<ProteinID>Q6IBQ3</ProteinID>
<Longitud>364</Longitud>
</Protein>
</Proteinas>
<NumeroDeResultados>2</NumeroDeResultados>
</ResultadoDeBusqueda>'

# Lee el Texto en formato XML
xml_document <- read_xml(xml_content)

# Lo convierte a una lista
xml_list <- as_list(xml_document)

In [ ]:
# Todos los datos que están en 'ResultadoDeBusqueda'
xml_list$ResultadoDeBusqueda

In [ ]:
# La cantidad de resultados
as.numeric(xml_list$ResultadoDeBusqueda$NumeroDeResultados)

In [ ]:
lista_de_proteinas <- xml_list$ResultadoDeBusqueda$Proteinas
print(paste("La lista tiene", length(lista_de_proteinas), "proteinas"))
lista_de_proteinas

In [ ]:
# La primer proteina
# xml_list$ResultadoDeBusqueda$Proteinas[[1]]
primer_proteina <- lista_de_proteinas[[1]]
primer_proteina

In [ ]:
# Los datos de la primer proteina
# trimws, borras los espacios en blando que hay al inicio y fin de un texto
primer_proteina$Nombre

In [ ]:
# Recupero uns lista con todos los Nombres
nombres <- c()
for (nombre in xml_list$ResultadoDeBusqueda$Proteinas) {
    nombres <- c(nombres, nombre$Nombre)
}
nombres

In [ ]:
# Lo mismo pero de otra forma
nombres <- sapply(
    xml_list$ResultadoDeBusqueda$Proteinas,
    function(x) {x$Nombre}
)
nombres

In [ ]:
# Convertir a dataframe
df <- as.data.frame(do.call(rbind, xml_list$ResultadoDeBusqueda$Proteinas))
df
# Otra forma
proteinas <- as.data.frame(t(sapply(xml_list$ResultadoDeBusqueda$Proteinas, unlist)))
proteinas


## JSON - Javascript Object Notation

Es otro formato para representar datos jerárquicos y arbitrarios.
Es más simple que XML.
No está pensado para ser leido directamente por un usuario.

```json
{
    "Proteinas":[
        {
            "Nombre": FUS,
            "ProteinID": Q6IBQ5,
            "Longitud": 526
        },
        {
            "Nombre": FIBP,
            "ProteinID": Q6IBQ3,
            "Longitud": 364
        }
    ],
    "NumeroDeResultados": 2
}
```

In [ ]:
# Para instalar rjson
install.packages("rjson")

In [ ]:
library("rjson")

In [ ]:
respuesta <- '{
    "Proteinas":[
        {
            "Nombre": "FUS",
            "ProteinID": "Q6IBQ5",
            "Longitud": 526
        },
        {
            "Nombre": "FIBP",
            "ProteinID": "Q6IBQ3",
            "Longitud": 364
        }
    ],
    "NumeroDeResultados": 2
}'
json_list <- fromJSON(respuesta)

In [ ]:
# Mostrar las proteinas
json_list$Proteinas

In [ ]:
# Mostrar 
json_list$NumeroDeResultados

In [ ]:
# Extraer los nombres de proteínas
nombres <- c()
for (nombre in json_list$Proteinas) {
    nombres <- c(nombres, trimws(nombre$Nombre))
}
nombres

In [ ]:
# Otra forma
nombres <- sapply(
    json_list$Proteinas,
    function(x) {x$Nombre}
)
nombres

In [ ]:
# Convertir a dataframe
json_data_frame <- as.data.frame(do.call(rbind, json_list$Proteinas))
json_data_frame

# Otra forma
proteinas <- as.data.frame(t(sapply(json_list$Proteinas, unlist)))
proteinas

## Texto tabular

Es simplemente un texto plano que representa una tabla.
Cada línea de texto representa una fila de la tabla, requiere especificar un caracter de separación para las columnas.

Los datos que se pueden representar no son arbitrarios, tienen que tener forma de tabla y no son jerárquicos.

```txt
Nombre  ProteinID Longitud
FUS Q6IBQ5 526
FIBP Q6IBQ3 364
```


In [ ]:
tabla <- "Nombre  ProteinID Longitud
FUS Q6IBQ5 526
FIBP Q6IBQ3 364"

data <- read.table(text=tabla, header=TRUE)
data